In [1]:
from PIL import Image
from os.path import join
import imageio
from torch import nn
from torch.nn.modules.linear import Linear
from torch.utils.data import Dataset
from torchvision import transforms
from tqdm import tqdm
import numpy as np
import os,sys,os.path
import pandas as pd
import pickle
import pydicom
import glob
import collections
import pprint
import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms.functional as TF
import warnings
import tarfile
import zipfile
import random
from skimage.io import imread, imsave
from skimage.transform import resize
import hashlib
import pickle
from tqdm import tqdm
import time
import math
from torch.utils.data import Dataset
import argparse
from sklearn.cluster import KMeans
import scipy
import torch.nn as nn
import torch.optim as optim
import copy
from scipy.optimize import linear_sum_assignment as linear_assignment
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from tensorboardX import SummaryWriter

In [2]:
from option import args_parser
from models import CIFAR_VAE
from utils import save_decoded_image,Accuracy,allocatePairs
from sampling import Dataset_IIDsampling, LocalDataset, LocalDataloaders, average_weights, Dataset_non_IIDsampling, get_dataset_cifar10_extr_noniid,partition_data,LocalDataloaders_sample
from Localupdate_VAE import LocalUpdate

In [3]:
args = args_parser()
args.tag = 'federated'

In [4]:
torch.set_default_dtype(torch.float64)
print(torch.__version__)
torch.cuda.is_available()
device = torch.device("cuda:0")
print(device)
args = args_parser()
np.set_printoptions(threshold=np.inf)
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3,4,5,6,7'

1.9.0+rocm4.0.1
cuda:0


In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [6]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)

Files already downloaded and verified


In [9]:
args.num_users = 10
# trainset, testset, dict_users, dict_users_test = get_dataset_cifar10_extr_noniid(args.num_users, 5, 500, 1.0, rand_seed = 10,all_class = False, all_class_prop = 0.05)
train_dataset,testset, dict_users, traindata_cls_counts = partition_data(n_users = args.num_users, alpha=0.5,rand_seed = 0)


Files already downloaded and verified
Files already downloaded and verified


In [55]:
Loaders_train = LocalDataloaders_sample(train_dataset,dict_users,args.batch_size,ShuffleorNot = True, mini=0.4)
Major_need_classes = []
Major_classes = []
Total_classes = []
for idx in range(args.num_users):
    counts = [0,0,0,0,0,0,0,0,0,0]
    for batch_idx,(X,y) in enumerate(Loaders_train[idx]):
        batch = len(y)
        y = np.array(y)
        for i in range(batch):
            counts[int(y[i])] += 1
    counts = np.array(counts)
    print(counts)
    major_need_classes = counts.argsort()[::-1][7:]
    major_classes = counts.argsort()[::-1][0:3]
    Total_classes.append(counts/np.sum(counts))
    Major_need_classes.append(major_need_classes)
    Major_classes.append(major_classes)

[369  71  54  28 288 340   0 972   0   0]
[  7  12 381 625  41  70   0 289  44   6]
[  2   3 111 302 319 134  46 429 405  23]
[176 498  10  18 357 292  44   5 505 113]
[ 194 1191  227  284   37  142    0    0    0    0]
[497  23 144  54 623   2   1  24 153   0]
[  3 114 685  10   1  38 756  12 461   0]
[302   3 113 293   4 356 257   0  36 366]
[ 442   59  265  211  287  458   50   15    1 1443]
[  0  71  46 146  34 158 822 259 400  34]


In [11]:
print(Major_need_classes)
print(Major_classes)

[array([9, 8, 6]), array([0, 9, 6]), array([9, 0, 1]), array([3, 2, 7]), array([8, 7, 6]), array([5, 9, 6]), array([0, 9, 4]), array([4, 1, 7]), array([6, 7, 8]), array([9, 2, 0])]
[array([7, 0, 5]), array([3, 2, 7]), array([7, 8, 4]), array([8, 1, 4]), array([1, 3, 2]), array([4, 0, 8]), array([6, 2, 8]), array([5, 9, 3]), array([9, 5, 0]), array([6, 8, 7])]


In [12]:
from scipy.stats import wasserstein_distance
L2D = np.zeros((args.num_users,args.num_users))
for i in range(args.num_users):
    for j in range(args.num_users):
        common = np.intersect1d(Major_need_classes[i], Major_classes[j])
        L2D[i,j] = len(common)
print(L2D)

[[0. 0. 1. 1. 0. 1. 2. 1. 1. 2.]
 [1. 0. 0. 0. 0. 1. 1. 1. 2. 1.]
 [1. 0. 0. 1. 1. 1. 0. 1. 2. 0.]
 [1. 3. 1. 0. 2. 0. 1. 1. 0. 1.]
 [1. 1. 2. 1. 0. 1. 2. 0. 0. 3.]
 [1. 0. 0. 0. 0. 0. 1. 2. 2. 1.]
 [1. 0. 1. 1. 0. 2. 0. 1. 2. 0.]
 [1. 1. 2. 2. 1. 1. 0. 0. 0. 1.]
 [1. 1. 2. 1. 0. 1. 2. 0. 0. 3.]
 [1. 1. 0. 0. 1. 1. 1. 1. 2. 0.]]


In [13]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [14]:
global_model = CIFAR_VAE(in_channels=3,input_shape=[32,32],code_length= 128, classes = 10,drop_rate=0.2,decoder = True)

In [15]:
print('# model parameters:', sum(param.numel() for param in global_model.parameters()))

# model parameters: 1840340


In [16]:
# use multi-GPU to train parallelly
global_model = nn.DataParallel(global_model)
global_model.to(device)
for m in global_model.modules():
    if isinstance(m, (nn.Conv2d, nn.Linear)):
        nn.init.xavier_uniform_(m.weight, gain=nn.init.calculate_gain('relu'))

In [17]:
optimizer = optim.Adam(global_model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=args.lr_sh_rate, gamma=0.5)

In [18]:
logger = SummaryWriter('../logs')
checkpoint_dir = './checkpoint/'+args.tag+'/VAE_CIFAR/'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
with open(checkpoint_dir+'args.pkl', 'wb') as fp:
    pickle.dump(args, fp)
print('Data and model loaded')
print('Checkpoint dir:', checkpoint_dir)

Data and model loaded
Checkpoint dir: ./checkpoint/centralized/VAE_CIFAR/


In [19]:
# copy weights
global_weights = global_model.state_dict()

In [20]:
# pre-training
args.num_epochs = 20
train_loss, train_accuracy = [], []
val_acc_list, net_list = [], []
cv_loss, cv_acc = [], []
print_every = 2
val_loss_pre, counter = 0, 0
for epoch in tqdm(range(args.num_epochs)):
    local_weights, local_losses = [], []
    print(f'\n | Global Training Round : {epoch+1} |\n')

    global_model.train()
    m = max(int(args.update_frac * args.num_users), 1)
    idxs_users = np.random.choice(range(args.num_users), m, replace=False)
    for idx in range(args.num_users):
        local_model = LocalUpdate(args,Loaders_train[idx], idxs=dict_users[idx], logger=logger)
        w, loss = local_model.update_weights(
            model=copy.deepcopy(global_model), global_round=epoch, u=0.1, device=device)
        local_weights.append(copy.deepcopy(w))
        local_losses.append(copy.deepcopy(loss))


    # update global weights
    global_weights = average_weights(local_weights)

    # update global weights
    global_model.load_state_dict(global_weights)

    loss_avg = sum(local_losses) / len(local_losses)
    train_loss.append(loss_avg)
    print("average loss:  ", loss_avg)

  0%|          | 0/20 [00:00<?, ?it/s]/home/huancheng/miniconda3/envs/anomaly/lib/python3.6/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



 | Global Training Round : 1 |



/home/huancheng/miniconda3/envs/anomaly/lib/python3.6/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


| Global Round : 0 | Local Epoch : 0 | [0/2122 (0%)]KLD: 2.204731 BCE: 2290.297657 Acc: 0.109375
| Global Round : 0 | Local Epoch : 0 | [640/2122 (29%)]KLD: 6.421499 BCE: 2113.947553 Acc: 0.390625
| Global Round : 0 | Local Epoch : 0 | [1280/2122 (59%)]KLD: 3.467060 BCE: 2099.229003 Acc: 0.375000
| Global Round : 0 | Local Epoch : 0 | [1920/2122 (88%)]KLD: 3.173329 BCE: 2082.472990 Acc: 0.531250
| Global Round : 0 | Local Epoch : 1 | [0/2122 (0%)]KLD: 3.138604 BCE: 2080.988488 Acc: 0.421875
| Global Round : 0 | Local Epoch : 1 | [640/2122 (29%)]KLD: 3.397675 BCE: 2076.214730 Acc: 0.656250
| Global Round : 0 | Local Epoch : 1 | [1280/2122 (59%)]KLD: 2.968647 BCE: 2074.271898 Acc: 0.656250
| Global Round : 0 | Local Epoch : 1 | [1920/2122 (88%)]KLD: 2.679238 BCE: 2081.930469 Acc: 0.718750
| Global Round : 0 | Local Epoch : 2 | [0/2122 (0%)]KLD: 2.772875 BCE: 2061.791858 Acc: 0.875000
| Global Round : 0 | Local Epoch : 2 | [640/2122 (29%)]KLD: 2.863972 BCE: 2067.109672 Acc: 0.703125
| Glo

  5%|▌         | 1/20 [02:01<38:26, 121.42s/it]

...

| Global Round : 17 | Local Epoch : 4 | [1500/1970 (97%)]KLD: 2.254427 BCE: 2051.289409 Acc: 0.920000
average loss:   205.3801766821441

 | Global Training Round : 19 |

| Global Round : 18 | Local Epoch : 0 | [0/2122 (0%)]KLD: 2.145662 BCE: 2049.974402 Acc: 0.671875
| Global Round : 18 | Local Epoch : 0 | [640/2122 (29%)]KLD: 2.132014 BCE: 2047.226734 Acc: 0.812500
| Global Round : 18 | Local Epoch : 0 | [1280/2122 (59%)]KLD: 2.109608 BCE: 2043.861935 Acc: 0.843750
| Global Round : 18 | Local Epoch : 0 | [1920/2122 (88%)]KLD: 2.142007 BCE: 2057.329791 Acc: 0.921875
| Global Round : 18 | Local Epoch : 1 | [0/2122 (0%)]KLD: 2.188848 BCE: 2041.945848 Acc: 0.906250
| Global Round : 18 | Local Epoch : 1 | [640/2122 (29%)]KLD: 2.125597 BCE: 2049.858830 Acc: 0.890625
| Global Round : 18 | Local Epoch : 1 | [1280/2122 (59%)]KLD: 2.147295 BCE: 2057.130502 Acc: 0.937500
| Global Round : 18 | Local Epoch : 1 | [1920/2122 (88%)]KLD: 2.235341 BCE: 2048.610480 Acc: 0.968750
| Global Round : 18 | Lo

 95%|█████████▌| 19/20 [33:16<01:44, 104.44s/it]

| Global Round : 18 | Local Epoch : 4 | [1500/1970 (97%)]KLD: 2.269086 BCE: 2046.458609 Acc: 1.000000
average loss:   205.3628027146755

 | Global Training Round : 20 |

| Global Round : 19 | Local Epoch : 0 | [0/2122 (0%)]KLD: 2.189674 BCE: 2046.544288 Acc: 0.656250
| Global Round : 19 | Local Epoch : 0 | [640/2122 (29%)]KLD: 2.132090 BCE: 2047.293065 Acc: 0.796875
| Global Round : 19 | Local Epoch : 0 | [1280/2122 (59%)]KLD: 2.095637 BCE: 2047.200465 Acc: 0.828125
| Global Round : 19 | Local Epoch : 0 | [1920/2122 (88%)]KLD: 2.087171 BCE: 2061.644942 Acc: 0.859375
| Global Round : 19 | Local Epoch : 1 | [0/2122 (0%)]KLD: 2.101952 BCE: 2050.519724 Acc: 0.937500
| Global Round : 19 | Local Epoch : 1 | [640/2122 (29%)]KLD: 2.200873 BCE: 2054.007396 Acc: 1.000000
| Global Round : 19 | Local Epoch : 1 | [1280/2122 (59%)]KLD: 2.222557 BCE: 2042.613907 Acc: 0.968750
| Global Round : 19 | Local Epoch : 1 | [1920/2122 (88%)]KLD: 2.267808 BCE: 2047.748522 Acc: 0.968750
| Global Round : 19 | Lo

100%|██████████| 20/20 [35:00<00:00, 105.01s/it]

| Global Round : 19 | Local Epoch : 4 | [1500/1970 (97%)]KLD: 2.341226 BCE: 2045.797992 Acc: 1.000000
average loss:   205.33027069990007


In [21]:
from torchvision.utils import save_image
def save_decoded_image(img, name):
    img = img.view(img.size(0), 3, 32, 32)
    save_image(img, name)

In [22]:
Models = []
for idx in range(args.num_users):
    model = CIFAR_VAE(in_channels=3,input_shape=[32,32],code_length= 128, classes = 10,drop_rate=0.2,decoder = True)
    model = nn.DataParallel(model)
    model.load_state_dict(local_weights[idx])
    model.to(device)
    model.to(device)
    Models.append(model)

In [23]:
loader_test = torch.utils.data.DataLoader(testset, batch_size=256,shuffle=True, num_workers=2)
accuracy = 0
count = 0
global_model.eval()  
for cnt, (X,y) in enumerate(loader_test):
    X = X.to(device)
    m = torch.nn.Sigmoid()
    X = m(X)
    y = y.double().to(device)
    mu,logVar,p,_X = global_model(X)
    y_pred = p.argmax(1)
    accuracy += Accuracy(y,y_pred)
    count += 1
print("accuracy of test:",accuracy/(count-1))

accuracy of test: 0.6493389423076923


In [24]:
def reparameterize(mu, logVar):
    #Reparameterization takes in the input mu and logVar and sample the mu + std * eps
    std = torch.exp(logVar/2)
    eps = torch.randn_like(std)
    return mu + std * eps

In [25]:
accuracy = 0
count = 0
Models[0].eval()  
for cnt, (X,y) in enumerate(Loaders_train[0]):
    X = X.to(device)
    m = torch.nn.Sigmoid()
    X = m(X)
    y = y.double().to(device)
    mu,logVar,p,_ = Models[0](X)
    z = reparameterize(mu,logVar)
    _X = global_model.module.Decoder(z)
    y_pred = p.argmax(1)
    accuracy += Accuracy(y,y_pred)
    count += 1
    if cnt % 5 == 0:
        save_decoded_image(X.cpu().data, name='./Generated_Images/original{}.png'.format(cnt))
        save_decoded_image(_X.cpu().data, name='./Generated_Images/decoded{}.png'.format(cnt))
print("accuracy of test:",accuracy/count)

accuracy of test: 0.9834558823529411


In [26]:
def getLocalMean(model,trainloader,major_classes,device):
        data_len = len(trainloader)
        Mu = []
        Var = []
        for i in range(len(major_classes)):
            mu = []
            var = []
            Mu.append(mu)
            Var.append(var)
        for cnt, (X,y) in enumerate(trainloader):
            X = X.to(device)
            m = torch.nn.Sigmoid()
            X = m(X)
            mu,logVar,p,_ = model(X)
            var = logVar
            for i in range(len(mu)):
                for j in range(len(major_classes)):
                    if y[i] == major_classes[j]:
                        Mu[j].append(mu[i].detach().cpu().numpy())
                        Var[j].append(var[i].detach().cpu().numpy())
                        break
        
        for i in range(len(major_classes)):
            Mu[i] = torch.DoubleTensor(Mu[i])     
            Var[i] = torch.DoubleTensor(Var[i])  
    
        return Mu,Var

In [27]:
def getLocalMeans(model,Loaders_train,num_users,Major_classes,device):
    Means = []
    Vars = []
    for idx in range(num_users):
        Mean,Var = getLocalMean(model,Loaders_train[idx],Major_classes[idx],device)
        Means.append(Mean) 
        Vars.append(Var)    
    return Means,Vars

In [36]:
from keras.utils import to_categorical
from sklearn.cluster import KMeans
from collections import Counter
class GenData(Dataset):
    def __init__(self, 
                 model,
                 global_model,
                 Mean,
                 Var,
                 major_classes, 
                 num_gen,
                 device,
                 idx):
        super(GenData, self).__init__()
        self.model = model.to(device)
        self.Mean = Mean
        self.Var = Var
        self.num_gen = num_gen
        X = []
        y = []
        latency = []
        print(major_classes)
        m = len(major_classes)
        num_sample = 10
        for i in range(m):
            z = []
            for j in range(len(Mean[i])):
                lantency = reparameterize(Mean[i][j],Var[i][j])
                z.append(lantency)
                
            z = torch.stack(z).cpu().detach().numpy() 
            num = int(z.shape[0]/2)
            z1 = z[0:num]
            z2 = z[num:2*num]
            z = []
            for k in range(num_sample):
                z_inter = z1*(k+1)/11 + z2*(10 - k)/11
                z.extend(z_inter)

            z = torch.DoubleTensor(z).to(device)        
            X = global_model.module.Decoder(z)
            mu,logVar,p,_ = model(X)
            p = torch.sigmoid(p)
            c = np.argmax(p.cpu().detach().numpy(),1)
            p = torch.max(p,dim=1).values
            for j in range(num*num_sample):
                if c[j] == major_classes[i] and p[j] >= 0.95:
                    latency.append(z[j])
                    y.append(major_classes[i])
        if len(latency) > 500:
            index_range = range(len(latency))
            sample_idex = np.random.choice(index_range,size=500)
            latency = [latency[x] for x in sample_idex] 
            y = [y[x] for x in sample_idex] 
        latency = torch.stack(latency).to(device)
        X = global_model.module.Decoder(latency).detach()
        save_decoded_image(X.cpu().data, name='./Generated_Images/X{}.png'.format(idx))
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):      
        return self.X[idx],self.y[idx]

In [44]:
print("Generate new dataset...")
# Loaders_train = LocalDataloaders(trainset,dict_users,args.batch_size,ShuffleorNot = True)
Loaders_train = LocalDataloaders_sample(trainset,dict_users,args.batch_size,ShuffleorNot = True, mini=0.4)
Means,Vars = getLocalMeans(copy.deepcopy(global_model),copy.deepcopy(Loaders_train),args.num_users,Major_classes,device)
loaders_gen = []
for idx in range(args.num_users):
    match_index = np.argmax(L2D[idx])
    gen_num = int(len(dict_users[idx]))
    dataset_gen = GenData(copy.deepcopy(global_model),copy.deepcopy(global_model),Means[match_index],Vars[match_index],Major_classes[match_index],gen_num,device,idx)
    loader_gen = torch.utils.data.DataLoader(
                    dataset_gen,
                    batch_size=args.batch_size,
                    shuffle =True)
    loaders_gen.append(loader_gen)      
#     loaders_gen.append(Loaders_train[match_index])      
print("Generation done...")

Generate new dataset...
[6 2 8]
[9 5 0]
[9 5 0]
[3 2 7]
[6 8 7]
[5 9 0]
[4 0 8]
[8 7 4]
[6 8 7]
[9 5 0]
Generation done...


In [45]:
class ConcatDataset(Dataset):
    def __init__(self, dataloader1, dataloader2):
        for idx,(X, y) in enumerate(dataloader1):
            m = torch.nn.Sigmoid()
            X = m(X)
            if idx == 0: 
                X0 = X.to(device)
                y0 = y.to(device)
            else:
                X0 = torch.cat((X0,X.to(device)),dim = 0)
                y0 = torch.cat((y0,y.to(device)),dim = 0)
                
        for idx,(X, y) in enumerate(dataloader2):
            X0 = torch.cat((X0,X.to(device)),dim = 0)
            y0 = torch.cat((y0,y.to(device)),dim = 0)
            
        self.X = X0
        self.y = y0


    def __getitem__(self, idx):
        return self.X[idx],self.y[idx]

    def __len__(self):
        return len(self.y)

In [46]:
Merge_loaders = []
for idx in range(args.num_users):
    merge_dataset = ConcatDataset(Loaders_train[idx],loaders_gen[idx])
    merge_loader = torch.utils.data.DataLoader(merge_dataset, batch_size=args.batch_size,shuffle=True)
    Merge_loaders.append(merge_loader)

In [48]:
Major_classes = []
Total_classes = []
for idx in range(args.num_users):
    counts = [0,0,0,0,0,0,0,0,0,0]
    for batch_idx,(X,y) in enumerate(Merge_loaders[idx]):
        y = y.cpu()
        batch = len(y)
        y = np.array(y)
        for i in range(batch):
            counts[int(y[i])] += 1
    counts = np.array(counts)
    print(counts)
    major_classes = counts.argsort()[::-1][0:3]
    Total_classes.append(counts/np.sum(counts))
    Major_classes.append(major_classes)

[ 381   54  321   30  272  313    4 1011  236    0]
[147  15 355 666  50 428   0 269  37   8]
[132   1 108 318 315 473  44 442 420  21]
[166 477 186 342 331 280  42  18 570 106]
[ 206 1167  231  251   59  161    7   37  456    0]
[624  34 123  42 652 357   1  28 156   4]
[114 118 652  18 283  66 766  14 549   0]
[282   1 129 299 157 394 223  31 354 360]
[ 439   69  269  197  297  464   43   45  458 1450]
[123  81  43 122  26 538 887 252 341  57]


In [49]:
new_model = copy.deepcopy(global_model)
optimizer = optim.Adam(new_model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=args.lr_sh_rate, gamma=0.5)

In [50]:
# training
args.num_epochs = 30
train_loss, train_accuracy = [], []
val_acc_list, net_list = [], []
cv_loss, cv_acc = [], []
print_every = 2
val_loss_pre, counter = 0, 0
for epoch in tqdm(range(args.num_epochs)):
    local_weights, local_losses = [], []
    print(f'\n | Global Training Round : {epoch+1} |\n')

    new_model.train()
    m = max(int(args.update_frac * args.num_users), 1)
    idxs_users = np.random.choice(range(args.num_users), m, replace=False)
    for idx in range(args.num_users):
        local_model = LocalUpdate(args, Merge_loaders[idx], idxs=dict_users[idx], logger=logger)
        w, loss = local_model.update_weights_norm(
            model=copy.deepcopy(new_model), global_round=epoch, u=0.1, device=device)
        local_weights.append(copy.deepcopy(w))
        local_losses.append(copy.deepcopy(loss))


    # update global weights
    global_weights = average_weights(local_weights)

    # update global weights
    new_model.load_state_dict(global_weights)

    loss_avg = sum(local_losses) / len(local_losses)
    train_loss.append(loss_avg)
    print("average loss:  ", loss_avg)

  0%|          | 0/30 [00:00<?, ?it/s]/home/huancheng/miniconda3/envs/anomaly/lib/python3.6/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



 | Global Training Round : 1 |

| Global Round : 0 | Local Epoch : 0 | [0/2622 (0%)]KLD: 2.118235 BCE: 2057.148488 Acc: 0.671875
| Global Round : 0 | Local Epoch : 0 | [640/2622 (24%)]KLD: 2.022562 BCE: 2067.817513 Acc: 0.796875
| Global Round : 0 | Local Epoch : 0 | [1280/2622 (49%)]KLD: 2.017058 BCE: 2056.457987 Acc: 0.796875
| Global Round : 0 | Local Epoch : 0 | [1920/2622 (73%)]KLD: 1.971514 BCE: 2057.648970 Acc: 0.796875
| Global Round : 0 | Local Epoch : 0 | [2480/2622 (98%)]KLD: 2.001145 BCE: 2065.666532 Acc: 0.774194
| Global Round : 0 | Local Epoch : 1 | [0/2622 (0%)]KLD: 1.992431 BCE: 2059.070428 Acc: 0.906250
| Global Round : 0 | Local Epoch : 1 | [640/2622 (24%)]KLD: 1.995936 BCE: 2047.711638 Acc: 0.828125
| Global Round : 0 | Local Epoch : 1 | [1280/2622 (49%)]KLD: 1.967629 BCE: 2057.020979 Acc: 0.890625
| Global Round : 0 | Local Epoch : 1 | [1920/2622 (73%)]KLD: 2.016213 BCE: 2057.245965 Acc: 0.875000
| Global Round : 0 | Local Epoch : 1 | [2480/2622 (98%)]KLD: 2.08153

  3%|▎         | 1/30 [01:54<55:11, 114.17s/it]

average loss:   206.18505379656708

 | Global Training Round : 2 |

| Global Round : 1 | Local Epoch : 0 | [0/2622 (0%)]KLD: 2.000292 BCE: 2055.995277 Acc: 0.718750
| Global Round : 1 | Local Epoch : 0 | [640/2622 (24%)]KLD: 2.019316 BCE: 2051.900625 Acc: 0.812500
| Global Round : 1 | Local Epoch : 0 | [1280/2622 (49%)]KLD: 2.026524 BCE: 2060.702509 Acc: 0.781250
| Global Round : 1 | Local Epoch : 0 | [1920/2622 (73%)]KLD: 1.855286 BCE: 2065.491950 Acc: 0.890625
| Global Round : 1 | Local Epoch : 0 | [2480/2622 (98%)]KLD: 1.989813 BCE: 2059.793781 Acc: 0.854839
| Global Round : 1 | Local Epoch : 1 | [0/2622 (0%)]KLD: 2.047704 BCE: 2054.305584 Acc: 0.843750
| Global Round : 1 | Local Epoch : 1 | [640/2622 (24%)]KLD: 1.912420 BCE: 2060.554515 Acc: 0.843750
| Global Round : 1 | Local Epoch : 1 | [1280/2622 (49%)]KLD: 2.113757 BCE: 2054.427032 Acc: 0.859375
| Global Round : 1 | Local Epoch : 1 | [1920/2622 (73%)]KLD: 2.002996 BCE: 2055.112475 Acc: 0.875000
| Global Round : 1 | Local Epoch 

  7%|▋         | 2/30 [03:46<52:40, 112.87s/it]

...

average loss:   205.77259795795345

 | Global Training Round : 29 |

| Global Round : 28 | Local Epoch : 0 | [0/2622 (0%)]KLD: 1.902012 BCE: 2057.233385 Acc: 0.796875
| Global Round : 28 | Local Epoch : 0 | [640/2622 (24%)]KLD: 1.910627 BCE: 2061.000916 Acc: 0.843750
| Global Round : 28 | Local Epoch : 0 | [1280/2622 (49%)]KLD: 1.860034 BCE: 2055.634745 Acc: 0.906250
| Global Round : 28 | Local Epoch : 0 | [1920/2622 (73%)]KLD: 2.032389 BCE: 2060.363860 Acc: 0.937500
| Global Round : 28 | Local Epoch : 0 | [2480/2622 (98%)]KLD: 1.932405 BCE: 2053.626001 Acc: 0.887097
| Global Round : 28 | Local Epoch : 1 | [0/2622 (0%)]KLD: 1.940585 BCE: 2053.988421 Acc: 0.953125
| Global Round : 28 | Local Epoch : 1 | [640/2622 (24%)]KLD: 1.945217 BCE: 2057.638690 Acc: 1.000000
| Global Round : 28 | Local Epoch : 1 | [1280/2622 (49%)]KLD: 1.945858 BCE: 2054.981151 Acc: 0.937500
| Global Round : 28 | Local Epoch : 1 | [1920/2622 (73%)]KLD: 1.966024 BCE: 2063.045445 Acc: 0.921875
| Global Round : 28 | L

 97%|█████████▋| 29/30 [54:39<01:53, 113.48s/it]

average loss:   205.7709289657236

 | Global Training Round : 30 |

| Global Round : 29 | Local Epoch : 0 | [0/2622 (0%)]KLD: 2.008457 BCE: 2060.532125 Acc: 0.750000
| Global Round : 29 | Local Epoch : 0 | [640/2622 (24%)]KLD: 1.877845 BCE: 2058.011570 Acc: 0.921875
| Global Round : 29 | Local Epoch : 0 | [1280/2622 (49%)]KLD: 1.875076 BCE: 2068.917944 Acc: 0.984375
| Global Round : 29 | Local Epoch : 0 | [1920/2622 (73%)]KLD: 1.942180 BCE: 2053.588299 Acc: 0.843750
| Global Round : 29 | Local Epoch : 0 | [2480/2622 (98%)]KLD: 1.911289 BCE: 2055.710618 Acc: 0.935484
| Global Round : 29 | Local Epoch : 1 | [0/2622 (0%)]KLD: 1.874244 BCE: 2067.142315 Acc: 0.953125
| Global Round : 29 | Local Epoch : 1 | [640/2622 (24%)]KLD: 1.991455 BCE: 2053.970424 Acc: 0.984375
| Global Round : 29 | Local Epoch : 1 | [1280/2622 (49%)]KLD: 2.002462 BCE: 2048.281973 Acc: 0.984375
| Global Round : 29 | Local Epoch : 1 | [1920/2622 (73%)]KLD: 1.938070 BCE: 2062.542634 Acc: 0.984375
| Global Round : 29 | Lo

100%|██████████| 30/30 [56:33<00:00, 113.10s/it]

average loss:   205.76406602497954


In [51]:
loader_test = torch.utils.data.DataLoader(testset, batch_size=256,shuffle=True, num_workers=2)
accuracy = 0
count = 0
new_model.eval()  
for cnt, (X,y) in enumerate(loader_test):
    X = X.to(device)
    m = torch.nn.Sigmoid()
    X = m(X)
    y = y.double().to(device)
    mu,logVar,p,_X = new_model(X)
    y_pred = p.argmax(1)
    accuracy += Accuracy(y,y_pred)
    count += 1
print("accuracy of test:",accuracy/(count-1))

accuracy of test: 0.6900040064102564


argument_interpolation_all

In [52]:
new_model_all_all = copy.deepcopy(global_model)
optimizer = optim.Adam(new_model_all_all.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=args.lr_sh_rate, gamma=0.5)

In [56]:
# training
args.num_epochs = 30
train_loss, train_accuracy = [], []
val_acc_list, net_list = [], []
cv_loss, cv_acc = [], []
print_every = 2
val_loss_pre, counter = 0, 0
for epoch in tqdm(range(args.num_epochs)):
    local_weights, local_losses = [], []
    print(f'\n | Global Training Round : {epoch+1} |\n')

    new_model_all_all.train()
    
    
    print("Generate new dataset...")
    Pairs = allocatePairs(args.num_users)
    Means,Vars = getLocalMeans(new_model_all_all,Loaders_train,args.num_users,Major_classes,device)
    loaders_gen = []
    if np.mod(epoch,5) == 0:
        L2D_replication = np.copy(L2D)
        
    for idx in range(args.num_users):
        match_index = np.argmax(L2D_replication[idx])
        L2D_replication[idx][match_index] = 0
        gen_num = int(len(dict_users[idx]))
        dataset_gen = GenData(new_model_all_all,new_model_all_all,Means[match_index],Vars[match_index],Major_classes[match_index],gen_num,device,idx)
        loader_gen = torch.utils.data.DataLoader(
                        dataset_gen,
                        batch_size=args.batch_size,
                        shuffle =True)
        loaders_gen.append(loader_gen)      
    print("Generation done...")
    
    Merge_loaders = []
    for idx in range(args.num_users):
        merge_dataset = ConcatDataset(Loaders_train[idx],loaders_gen[idx])
        merge_loader = torch.utils.data.DataLoader(merge_dataset, batch_size=args.batch_size,shuffle=True)
        Merge_loaders.append(merge_loader)

    m = max(int(args.update_frac * args.num_users), 1)
    idxs_users = np.random.choice(range(args.num_users), m, replace=False)
    for idx in range(args.num_users):
        local_model = LocalUpdate(args, Merge_loaders[idx], idxs=dict_users[idx], logger=logger)
        w, loss = local_model.update_weights_norm(
            model=copy.deepcopy(new_model_all_all), global_round=epoch, u=0.1, device=device)
        local_weights.append(copy.deepcopy(w))
        local_losses.append(copy.deepcopy(loss))


    # update global weights
    global_weights = average_weights(local_weights)

    # update global weights
    new_model_all_all.load_state_dict(global_weights)

    loss_avg = sum(local_losses) / len(local_losses)
    train_loss.append(loss_avg)
    print("average loss:  ", loss_avg)

  0%|          | 0/30 [00:00<?, ?it/s]


 | Global Training Round : 1 |

Generate new dataset...
[6 2 8]
[9 5 0]
[9 5 0]
[3 2 7]
[6 8 7]
[9 5 0]
[4 0 8]
[7 8 4]
[6 8 7]
[9 5 0]
Generation done...


/home/huancheng/miniconda3/envs/anomaly/lib/python3.6/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


| Global Round : 0 | Local Epoch : 0 | [0/2622 (0%)]KLD: 2.169175 BCE: 2057.092719 Acc: 0.656250
| Global Round : 0 | Local Epoch : 0 | [640/2622 (24%)]KLD: 2.007128 BCE: 2050.788308 Acc: 0.734375
| Global Round : 0 | Local Epoch : 0 | [1280/2622 (49%)]KLD: 2.030149 BCE: 2058.419293 Acc: 0.812500
| Global Round : 0 | Local Epoch : 0 | [1920/2622 (73%)]KLD: 1.972361 BCE: 2059.589657 Acc: 0.781250
| Global Round : 0 | Local Epoch : 0 | [2480/2622 (98%)]KLD: 2.009106 BCE: 2053.138734 Acc: 0.774194
| Global Round : 0 | Local Epoch : 1 | [0/2622 (0%)]KLD: 2.023068 BCE: 2055.352912 Acc: 0.875000
| Global Round : 0 | Local Epoch : 1 | [640/2622 (24%)]KLD: 2.016012 BCE: 2050.148821 Acc: 0.906250
| Global Round : 0 | Local Epoch : 1 | [1280/2622 (49%)]KLD: 2.104342 BCE: 2042.819874 Acc: 0.875000
| Global Round : 0 | Local Epoch : 1 | [1920/2622 (73%)]KLD: 2.117414 BCE: 2053.165898 Acc: 0.921875
| Global Round : 0 | Local Epoch : 1 | [2480/2622 (98%)]KLD: 2.189339 BCE: 2050.795638 Acc: 0.838710


  3%|▎         | 1/30 [02:37<1:16:02, 157.34s/it]

...

average loss:   205.3778061500569

 | Global Training Round : 29 |

Generate new dataset...
[8 1 4]
[6 2 8]
[1 3 2]
[7 8 4]
[7 0 5]
[6 2 8]
[7 8 4]
[3 2 7]
[7 0 5]
[1 3 2]
Generation done...
| Global Round : 28 | Local Epoch : 0 | [0/2622 (0%)]KLD: 1.990310 BCE: 2047.528582 Acc: 0.796875
| Global Round : 28 | Local Epoch : 0 | [640/2622 (24%)]KLD: 2.014383 BCE: 2049.763481 Acc: 0.859375
| Global Round : 28 | Local Epoch : 0 | [1280/2622 (49%)]KLD: 2.046841 BCE: 2044.857225 Acc: 0.890625
| Global Round : 28 | Local Epoch : 0 | [1920/2622 (73%)]KLD: 1.979964 BCE: 2051.509641 Acc: 0.953125
| Global Round : 28 | Local Epoch : 0 | [2480/2622 (98%)]KLD: 1.967454 BCE: 2058.503111 Acc: 0.935484
| Global Round : 28 | Local Epoch : 1 | [0/2622 (0%)]KLD: 1.954924 BCE: 2058.205009 Acc: 0.953125
| Global Round : 28 | Local Epoch : 1 | [640/2622 (24%)]KLD: 1.973384 BCE: 2044.914971 Acc: 0.953125
| Global Round : 28 | Local Epoch : 1 | [1280/2622 (49%)]KLD: 2.090876 BCE: 2041.924203 Acc: 1.000000
| G

 97%|█████████▋| 29/30 [1:17:03<02:43, 163.25s/it]

average loss:   205.30948018552652

 | Global Training Round : 30 |

Generate new dataset...
[4 0 8]
[9 5 0]
[4 0 8]
[6 2 8]
[3 2 7]
[6 8 7]
[8 1 4]
[1 3 2]
[3 2 7]
[4 0 8]
Generation done...
| Global Round : 29 | Local Epoch : 0 | [0/2622 (0%)]KLD: 2.031609 BCE: 2042.189409 Acc: 0.875000
| Global Round : 29 | Local Epoch : 0 | [640/2622 (24%)]KLD: 1.927047 BCE: 2048.014501 Acc: 0.937500
| Global Round : 29 | Local Epoch : 0 | [1280/2622 (49%)]KLD: 1.967410 BCE: 2052.652375 Acc: 0.921875
| Global Round : 29 | Local Epoch : 0 | [1920/2622 (73%)]KLD: 1.919886 BCE: 2047.725147 Acc: 0.921875
| Global Round : 29 | Local Epoch : 0 | [2480/2622 (98%)]KLD: 1.968251 BCE: 2053.108767 Acc: 0.887097
| Global Round : 29 | Local Epoch : 1 | [0/2622 (0%)]KLD: 1.972713 BCE: 2049.944170 Acc: 0.953125
| Global Round : 29 | Local Epoch : 1 | [640/2622 (24%)]KLD: 1.943156 BCE: 2060.227167 Acc: 1.000000
| Global Round : 29 | Local Epoch : 1 | [1280/2622 (49%)]KLD: 1.933322 BCE: 2055.435812 Acc: 0.953125
| 

100%|██████████| 30/30 [1:19:38<00:00, 159.30s/it]

average loss:   205.30186400239535


In [57]:
loader_test = torch.utils.data.DataLoader(testset, batch_size=256,shuffle=True, num_workers=2)
accuracy = 0
count = 0
new_model_all_all.eval()  
for cnt, (X,y) in enumerate(loader_test):
    X = X.to(device)
    m = torch.nn.Sigmoid()
    X = m(X)
    y = y.double().to(device)
    mu,logVar,p,_X = new_model_all_all(X)
    y_pred = p.argmax(1)
    accuracy += Accuracy(y,y_pred)
    count += 1
print("accuracy of test:",accuracy/(count-1))

accuracy of test: 0.6893028846153846


argument_groundtrue_eachround